In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

Using TensorFlow backend.


In [2]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 133)
    return files, targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('datasets/train')
valid_files, valid_targets = load_dataset('datasets/valid')
test_files, test_targets = load_dataset('datasets/test')

# load list of dog names
categories = [item for item in sorted(glob("datasets/train/*/"))]

# print statistics about the dataset
print('There are %d total categories.' % len(categories))
print('There are %s total images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training images.' % len(train_files))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

There are 3 total categories.
There are 21031 total images.

There are 13496 training images.
There are 3401 validation images.
There are 4134 test images.


In [3]:
img_width, img_height = 197, 197
nb_train_samples = len(train_files)
nb_validation_samples = len(valid_files)
batch_size = 32
epochs = 50

model = applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [4]:
for layer in model.layers[:-1]:
    layer.trainable = False

In [5]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)

In [7]:
# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [8]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

In [9]:
train_generator = train_datagen.flow_from_directory(
'datasets/train',
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
'datasets/valid',
target_size = (img_height, img_width),
class_mode = "categorical")

Found 13496 images belonging to 3 classes.
Found 3401 images belonging to 3 classes.


In [10]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("ResNet50_2.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
steps_per_epoch = nb_train_samples / batch_size,
epochs = epochs,
validation_data = validation_generator,
validation_steps = nb_validation_samples / batch_size,
callbacks = [checkpoint, early])

Epoch 1/50
356/421 [========================>.....] - ETA: 17s - loss: 1.2077 - acc: 0.3553

KeyboardInterrupt: 